# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 1:02PM,256953,10,9128061,Eywa Pharma Inc.,Released
1,Feb 8 2023 1:02PM,256953,10,9128062,Eywa Pharma Inc.,Released
2,Feb 8 2023 1:02PM,256953,10,9128070,Eywa Pharma Inc.,Released
3,Feb 8 2023 1:02PM,256953,10,9128071,Eywa Pharma Inc.,Released
4,Feb 8 2023 1:02PM,256953,10,9128125,Eywa Pharma Inc.,Released
5,Feb 8 2023 1:02PM,256953,10,9129826,Eywa Pharma Inc.,Released
6,Feb 8 2023 1:02PM,256953,10,9131836,Eywa Pharma Inc.,Released
7,Feb 8 2023 1:02PM,256953,10,9131837,Eywa Pharma Inc.,Released
8,Feb 8 2023 1:02PM,256953,10,9131839,Eywa Pharma Inc.,Released
9,Feb 8 2023 1:02PM,256953,10,9131840,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,256946,Released,15
48,256947,Released,1
49,256948,Released,1
50,256952,Executing,1
51,256953,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256946,NaN,NaN,15.0
256947,NaN,NaN,1.0
256948,NaN,NaN,1.0
256952,NaN,1.0,NaN
256953,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,37.0,8.0,7.0
256826,0.0,0.0,113.0
256827,12.0,1.0,3.0
256857,0.0,1.0,10.0
256868,4.0,17.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,37,8,7
256826,0,0,113
256827,12,1,3
256857,0,1,10
256868,4,17,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,37,8,7
1,256826,0,0,113
2,256827,12,1,3
3,256857,0,1,10
4,256868,4,17,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,37,8,7
1,256826,,,113
2,256827,12,1,3
3,256857,,1,10
4,256868,4,17,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 1:02PM,256953,10,Eywa Pharma Inc.
10,Feb 8 2023 12:57PM,256952,21,"NBTY Global, Inc."
11,Feb 8 2023 12:42PM,256945,10,ISDIN Corporation
33,Feb 8 2023 12:38PM,256946,10,Bio-PRF
48,Feb 8 2023 12:38PM,256948,10,Emerginnova
49,Feb 8 2023 12:34PM,256947,10,Bio-PRF
50,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation
54,Feb 8 2023 12:16PM,256940,10,Emerginnova
57,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC"
58,Feb 8 2023 12:07PM,256938,19,VITABIOTICS USA INC.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 1:02PM,256953,10,Eywa Pharma Inc.,,,10
1,Feb 8 2023 12:57PM,256952,21,"NBTY Global, Inc.",,1,
2,Feb 8 2023 12:42PM,256945,10,ISDIN Corporation,,,22
3,Feb 8 2023 12:38PM,256946,10,Bio-PRF,,,15
4,Feb 8 2023 12:38PM,256948,10,Emerginnova,,,1
5,Feb 8 2023 12:34PM,256947,10,Bio-PRF,,,1
6,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,,,4
7,Feb 8 2023 12:16PM,256940,10,Emerginnova,,,3
8,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC",,,1
9,Feb 8 2023 12:07PM,256938,19,VITABIOTICS USA INC.,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 1:02PM,256953,10,Eywa Pharma Inc.,10,,
1,Feb 8 2023 12:57PM,256952,21,"NBTY Global, Inc.",,1,
2,Feb 8 2023 12:42PM,256945,10,ISDIN Corporation,22,,
3,Feb 8 2023 12:38PM,256946,10,Bio-PRF,15,,
4,Feb 8 2023 12:38PM,256948,10,Emerginnova,1,,
5,Feb 8 2023 12:34PM,256947,10,Bio-PRF,1,,
6,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,4,,
7,Feb 8 2023 12:16PM,256940,10,Emerginnova,3,,
8,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC",1,,
9,Feb 8 2023 12:07PM,256938,19,VITABIOTICS USA INC.,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 1:02PM,256953,10,Eywa Pharma Inc.,10,,
1,Feb 8 2023 12:57PM,256952,21,"NBTY Global, Inc.",,1,
2,Feb 8 2023 12:42PM,256945,10,ISDIN Corporation,22,,
3,Feb 8 2023 12:38PM,256946,10,Bio-PRF,15,,
4,Feb 8 2023 12:38PM,256948,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 1:02PM,256953,10,Eywa Pharma Inc.,10.0,NaN,NaN
1,Feb 8 2023 12:57PM,256952,21,"NBTY Global, Inc.",NaN,1.0,NaN
2,Feb 8 2023 12:42PM,256945,10,ISDIN Corporation,22.0,NaN,NaN
3,Feb 8 2023 12:38PM,256946,10,Bio-PRF,15.0,NaN,NaN
4,Feb 8 2023 12:38PM,256948,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 1:02PM,256953,10,Eywa Pharma Inc.,10.0,0.0,0.0
1,Feb 8 2023 12:57PM,256952,21,"NBTY Global, Inc.",0.0,1.0,0.0
2,Feb 8 2023 12:42PM,256945,10,ISDIN Corporation,22.0,0.0,0.0
3,Feb 8 2023 12:38PM,256946,10,Bio-PRF,15.0,0.0,0.0
4,Feb 8 2023 12:38PM,256948,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5909092,108.0,11.0,3.0
15,1284271,136.0,31.0,49.0
16,256873,1.0,0.0,0.0
19,2055227,20.0,18.0,4.0
20,256930,1.0,0.0,0.0
21,513822,1.0,1.0,0.0
22,256918,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5909092,108.0,11.0,3.0
1,15,1284271,136.0,31.0,49.0
2,16,256873,1.0,0.0,0.0
3,19,2055227,20.0,18.0,4.0
4,20,256930,1.0,0.0,0.0
5,21,513822,1.0,1.0,0.0
6,22,256918,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,108.0,11.0,3.0
1,15,136.0,31.0,49.0
2,16,1.0,0.0,0.0
3,19,20.0,18.0,4.0
4,20,1.0,0.0,0.0
5,21,1.0,1.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,108.0
1,15,Released,136.0
2,16,Released,1.0
3,19,Released,20.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,3.0,49.0,0.0,4.0,0.0,0.0,0.0
Executing,11.0,31.0,0.0,18.0,0.0,1.0,0.0
Released,108.0,136.0,1.0,20.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,3.0,49.0,0.0,4.0,0.0,0.0,0.0
1,Executing,11.0,31.0,0.0,18.0,0.0,1.0,0.0
2,Released,108.0,136.0,1.0,20.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,3.0,49.0,0.0,4.0,0.0,0.0,0.0
1,Executing,11.0,31.0,0.0,18.0,0.0,1.0,0.0
2,Released,108.0,136.0,1.0,20.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()